In [1]:
#basic imports
import glob
import IPython
from random import randint

#data processing
import os, librosa
import numpy as np
from tqdm import tqdm
import scipy.io.wavfile
import python_speech_features as psf
from sklearn.preprocessing import scale
#modelling
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
import numpy as np
from sklearn import preprocessing
import python_speech_features as mfcc

In [3]:
IPython.display.Audio("dataset/1/00001.wav")

In [4]:
def load_audio_file(parent_dir,sub_dir, duration=3, sr=16000):
    for file in os.listdir(parent_dir + sub_dir):
        file_path = parent_dir + sub_dir + '/' + file
        input_length=sr*duration
        y, sr = librosa.load(file_path,sr=sr, duration=duration)
        dur = librosa.get_duration(y=y)
        if (round(dur) < duration):
            y = librosa.util.fix_length(y, round(input_length))      

        ps = librosa.feature.melspectrogram(y=y, sr=sr)
        label = file_path.split('/')[-2]
        return ps, label

In [5]:
parent_dir = 'dataset/'
sub_dirs = [ folder for folder in os.listdir(parent_dir)]
data = []
labels = []
for sub_dir in tqdm(sub_dirs):
#for sub_dir in sub_dirs:
    features, label = load_audio_file(parent_dir,sub_dir)
    data.append(features)
    labels.append(label)

100%|██████████████████████████████████████████████████████████████████████████████| 1211/1211 [01:02<00:00, 19.41it/s]


In [6]:
data = np.array(data)#, dtype='float32') / 255.0
labels = np.array(labels,dtype='int32' )
print(data.shape,labels.shape)

(1211, 128, 94) (1211,)


In [7]:
def make_pairs(data, labels):
    pairData = []
    pairLabels = []
    numClasses = len(np.unique(labels))
    idx = [np.where(labels == i)[0] for i in range(0, numClasses)]
    for idxA in range(len(data)):
        currentAudio = data[idxA]
        label = labels[idxA]
        idxB = np.random.choice(idx[label])
        posData = data[idxB]
        pairData.append([currentAudio, posData])
        pairLabels.append([1])
        negIdx = np.where(labels != label)[0]
        negData = data[np.random.choice(negIdx)]
        pairData.append([currentAudio, negData])
        pairLabels.append([0])
    return (np.array(pairData), np.array(pairLabels))

In [8]:
%%time
# prepare the positive and negative pairs
print("[INFO] preparing positive and negative pairs...")
(pairdata, pairlabels) = make_pairs(data, labels)

[INFO] preparing positive and negative pairs...
Wall time: 162 ms


In [9]:
trainx, testx, trainy, testy = train_test_split(pairdata, pairlabels, test_size=0.25, random_state=42)
print(trainx.shape, testx.shape, trainy.shape, testy.shape)

(1816, 2, 128, 94) (606, 2, 128, 94) (1816, 1) (606, 1)


In [47]:
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation,Subtract , MaxPooling2D
from tensorflow.keras.layers import Input, Lambda, Dense, Dropout, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.layers import LSTM #, CuDNNLSTM, GRU, CuDNNGRU
from tensorflow.keras.models import Model, Sequential
import tensorflow as tf

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)

In [48]:
def build_base_network(input_shape):
    input = Input(shape=input_shape)
    #x = Flatten()(input)
    x = LSTM(128, activation='relu')(input)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(64, activation='relu')(x)
    return Model(input, x)

In [49]:
input_dim = trainx.shape[2:]
print(input_dim)
audio_a = Input(shape=input_dim)
audio_b = Input(shape=input_dim)

(128, 94)


In [50]:
base_network = build_base_network(input_dim)

feat_vecs_a = base_network(audio_a)
feat_vecs_b = base_network(audio_b)

In [60]:
#distance = Lambda(euclidean_distance)([feat_vecs_a, feat_vecs_b])
#distance = Subtract()([feat_vecs_a, feat_vecs_b])
#distance = Lambda(lambda x: K.abs(x))(distance)
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import BatchNormalization

distance = concatenate([feat_vecs_a, feat_vecs_b])
merged = Dropout(0.2)(distance)
merged = Dense(128, activation='relu')(merged)

In [61]:
outputs = Dense(1, activation="sigmoid")(merged)
model = Model(inputs=[audio_a, audio_b], outputs=outputs)

In [62]:
model.compile(loss="binary_crossentropy", optimizer="rmsprop",metrics=["accuracy"])
# train the model
print("[INFO] training model...")
history = model.fit([trainx[:, 0], trainx[:, 1]], trainy[:],
                    validation_data=([testx[:, 0], testx[:, 1]], testy[:]),
                    batch_size=512, 
                    epochs=1000)

[INFO] training model...
Epoch 1/1000
4/4 [==============================] - 5s 759ms/step - loss: 9.7185 - accuracy: 0.5286 - val_loss: 1.5691 - val_accuracy: 0.6535
Epoch 2/1000
4/4 [==============================] - 2s 548ms/step - loss: 2.4141 - accuracy: 0.6327 - val_loss: 1.7655 - val_accuracy: 0.6964
Epoch 3/1000
4/4 [==============================] - 2s 560ms/step - loss: 2.0251 - accuracy: 0.6608 - val_loss: 1.4638 - val_accuracy: 0.7195
Epoch 4/1000
4/4 [==============================] - 2s 588ms/step - loss: 1.4036 - accuracy: 0.6883 - val_loss: 0.6466 - val_accuracy: 0.7508
Epoch 5/1000
4/4 [==============================] - 2s 576ms/step - loss: 1.8155 - accuracy: 0.7274 - val_loss: 1.1660 - val_accuracy: 0.7327
Epoch 6/1000
4/4 [==============================] - 2s 538ms/step - loss: 1.8982 - accuracy: 0.7483 - val_loss: 0.8535 - val_accuracy: 0.7888
Epoch 7/1000
4/4 [==============================] - 2s 587ms/step - loss: 1.0131 - accuracy: 0.7588 - val_loss: 1.7167 - va

4/4 [==============================] - 2s 570ms/step - loss: 0.4960 - accuracy: 0.8067 - val_loss: 0.6472 - val_accuracy: 0.8020
Epoch 59/1000
4/4 [==============================] - 2s 583ms/step - loss: 0.4881 - accuracy: 0.8122 - val_loss: 0.5460 - val_accuracy: 0.7904
Epoch 60/1000
4/4 [==============================] - 2s 542ms/step - loss: 0.5960 - accuracy: 0.8155 - val_loss: 0.5092 - val_accuracy: 0.8020
Epoch 61/1000
4/4 [==============================] - 2s 558ms/step - loss: 0.5584 - accuracy: 0.8199 - val_loss: 0.4147 - val_accuracy: 0.8020
Epoch 62/1000
4/4 [==============================] - 2s 535ms/step - loss: 0.6488 - accuracy: 0.8067 - val_loss: 0.6196 - val_accuracy: 0.8020
Epoch 63/1000
4/4 [==============================] - 2s 602ms/step - loss: 0.5654 - accuracy: 0.8100 - val_loss: 0.5600 - val_accuracy: 0.7937
Epoch 64/1000
4/4 [==============================] - 2s 570ms/step - loss: 0.4735 - accuracy: 0.8128 - val_loss: 0.3614 - val_accuracy: 0.8119
Epoch 65/1000

4/4 [==============================] - 2s 557ms/step - loss: 0.5147 - accuracy: 0.8227 - val_loss: 0.5523 - val_accuracy: 0.7921
Epoch 116/1000
4/4 [==============================] - 2s 533ms/step - loss: 0.4912 - accuracy: 0.8221 - val_loss: 0.3984 - val_accuracy: 0.8069
Epoch 117/1000
4/4 [==============================] - 2s 572ms/step - loss: 0.5646 - accuracy: 0.8188 - val_loss: 0.5230 - val_accuracy: 0.7937
Epoch 118/1000
4/4 [==============================] - 2s 543ms/step - loss: 0.4548 - accuracy: 0.8243 - val_loss: 0.3891 - val_accuracy: 0.8036
Epoch 119/1000
4/4 [==============================] - 2s 608ms/step - loss: 0.4377 - accuracy: 0.8254 - val_loss: 0.3710 - val_accuracy: 0.8102
Epoch 120/1000
4/4 [==============================] - 2s 556ms/step - loss: 0.4490 - accuracy: 0.8271 - val_loss: 0.3733 - val_accuracy: 0.8036
Epoch 121/1000
4/4 [==============================] - 2s 611ms/step - loss: 0.5118 - accuracy: 0.8232 - val_loss: 0.3642 - val_accuracy: 0.8036
Epoch 1

Epoch 172/1000
4/4 [==============================] - 2s 548ms/step - loss: 0.5420 - accuracy: 0.8282 - val_loss: 0.4149 - val_accuracy: 0.8168
Epoch 173/1000
4/4 [==============================] - 2s 536ms/step - loss: 0.4088 - accuracy: 0.8249 - val_loss: 0.3942 - val_accuracy: 0.8152
Epoch 174/1000
4/4 [==============================] - 2s 607ms/step - loss: 0.4155 - accuracy: 0.8276 - val_loss: 0.3430 - val_accuracy: 0.8185
Epoch 175/1000
4/4 [==============================] - 2s 542ms/step - loss: 0.3812 - accuracy: 0.8293 - val_loss: 0.4059 - val_accuracy: 0.8135
Epoch 176/1000
4/4 [==============================] - 2s 567ms/step - loss: 0.4480 - accuracy: 0.8271 - val_loss: 0.3962 - val_accuracy: 0.8119
Epoch 177/1000
4/4 [==============================] - 2s 539ms/step - loss: 0.4391 - accuracy: 0.8304 - val_loss: 0.3481 - val_accuracy: 0.8201
Epoch 178/1000
4/4 [==============================] - 2s 589ms/step - loss: 0.4005 - accuracy: 0.8298 - val_loss: 0.3502 - val_accuracy:

Epoch 229/1000
4/4 [==============================] - 2s 558ms/step - loss: 0.3948 - accuracy: 0.8265 - val_loss: 0.3502 - val_accuracy: 0.8135
Epoch 230/1000
4/4 [==============================] - 2s 550ms/step - loss: 0.3622 - accuracy: 0.8304 - val_loss: 0.6258 - val_accuracy: 0.8069
Epoch 231/1000
4/4 [==============================] - 2s 601ms/step - loss: 0.4891 - accuracy: 0.8271 - val_loss: 0.3681 - val_accuracy: 0.8201
Epoch 232/1000
4/4 [==============================] - 2s 536ms/step - loss: 0.4977 - accuracy: 0.8304 - val_loss: 0.3637 - val_accuracy: 0.8119
Epoch 233/1000
4/4 [==============================] - 2s 556ms/step - loss: 0.3932 - accuracy: 0.8348 - val_loss: 0.3297 - val_accuracy: 0.8234
Epoch 234/1000
4/4 [==============================] - 2s 535ms/step - loss: 0.4536 - accuracy: 0.8381 - val_loss: 0.3439 - val_accuracy: 0.8218
Epoch 235/1000
4/4 [==============================] - 2s 611ms/step - loss: 0.3385 - accuracy: 0.8403 - val_loss: 0.3359 - val_accuracy:

...

Epoch 742/1000
4/4 [==============================] - 2s 525ms/step - loss: 0.2324 - accuracy: 0.9235 - val_loss: 0.2911 - val_accuracy: 0.8795
Epoch 743/1000
4/4 [==============================] - 2s 567ms/step - loss: 0.2139 - accuracy: 0.9257 - val_loss: 0.3313 - val_accuracy: 0.8944
Epoch 744/1000
4/4 [==============================] - 2s 554ms/step - loss: 0.2631 - accuracy: 0.9246 - val_loss: 0.2643 - val_accuracy: 0.8927
Epoch 745/1000
4/4 [==============================] - 2s 547ms/step - loss: 0.2141 - accuracy: 0.9257 - val_loss: 0.2509 - val_accuracy: 0.8878
Epoch 746/1000
4/4 [==============================] - 2s 566ms/step - loss: 0.2162 - accuracy: 0.9257 - val_loss: 0.2420 - val_accuracy: 0.8878
Epoch 747/1000
4/4 [==============================] - 2s 548ms/step - loss: 0.2353 - accuracy: 0.9235 - val_loss: 0.2513 - val_accuracy: 0.8944
Epoch 748/1000
4/4 [==============================] - 2s 570ms/step - loss: 0.2473 - accuracy: 0.9235 - val_loss: 0.2400 - val_accuracy:

Epoch 799/1000
4/4 [==============================] - 2s 543ms/step - loss: 0.2156 - accuracy: 0.9246 - val_loss: 0.2590 - val_accuracy: 0.8911
Epoch 800/1000
4/4 [==============================] - 2s 567ms/step - loss: 0.2433 - accuracy: 0.9290 - val_loss: 0.3208 - val_accuracy: 0.8894
Epoch 801/1000
4/4 [==============================] - 2s 569ms/step - loss: 0.1990 - accuracy: 0.9284 - val_loss: 0.2591 - val_accuracy: 0.8894
Epoch 802/1000
4/4 [==============================] - 2s 552ms/step - loss: 0.2787 - accuracy: 0.9290 - val_loss: 0.3738 - val_accuracy: 0.8911
Epoch 803/1000
4/4 [==============================] - 2s 551ms/step - loss: 0.2045 - accuracy: 0.9284 - val_loss: 0.2556 - val_accuracy: 0.8894
Epoch 804/1000
4/4 [==============================] - 2s 550ms/step - loss: 0.2585 - accuracy: 0.9251 - val_loss: 0.2546 - val_accuracy: 0.8894
Epoch 805/1000
4/4 [==============================] - 2s 552ms/step - loss: 0.2422 - accuracy: 0.9306 - val_loss: 0.2380 - val_accuracy:

Epoch 856/1000
4/4 [==============================] - 2s 643ms/step - loss: 0.1933 - accuracy: 0.9328 - val_loss: 0.2323 - val_accuracy: 0.8960
Epoch 857/1000
4/4 [==============================] - 3s 663ms/step - loss: 0.2912 - accuracy: 0.9317 - val_loss: 0.2484 - val_accuracy: 0.8993
Epoch 858/1000
4/4 [==============================] - 2s 574ms/step - loss: 0.2098 - accuracy: 0.9290 - val_loss: 0.3249 - val_accuracy: 0.9010
Epoch 859/1000
4/4 [==============================] - 2s 571ms/step - loss: 0.2216 - accuracy: 0.9290 - val_loss: 0.2971 - val_accuracy: 0.8944
Epoch 860/1000
4/4 [==============================] - 2s 559ms/step - loss: 0.2035 - accuracy: 0.9295 - val_loss: 0.2329 - val_accuracy: 0.8911
Epoch 861/1000
4/4 [==============================] - 2s 624ms/step - loss: 0.2325 - accuracy: 0.9306 - val_loss: 0.2780 - val_accuracy: 0.8960
Epoch 862/1000
4/4 [==============================] - 2s 594ms/step - loss: 0.2148 - accuracy: 0.9301 - val_loss: 0.2254 - val_accuracy:

Epoch 913/1000
4/4 [==============================] - 2s 595ms/step - loss: 0.2165 - accuracy: 0.9356 - val_loss: 0.4151 - val_accuracy: 0.9026
Epoch 914/1000
4/4 [==============================] - 2s 589ms/step - loss: 0.2697 - accuracy: 0.9356 - val_loss: 0.2209 - val_accuracy: 0.9059
Epoch 915/1000
4/4 [==============================] - 2s 584ms/step - loss: 0.3178 - accuracy: 0.9389 - val_loss: 0.3280 - val_accuracy: 0.8911
Epoch 916/1000
4/4 [==============================] - 2s 614ms/step - loss: 0.2224 - accuracy: 0.9350 - val_loss: 0.2763 - val_accuracy: 0.9059
Epoch 917/1000
4/4 [==============================] - 2s 562ms/step - loss: 0.1796 - accuracy: 0.9378 - val_loss: 0.2648 - val_accuracy: 0.9026
Epoch 918/1000
4/4 [==============================] - 2s 595ms/step - loss: 0.2038 - accuracy: 0.9372 - val_loss: 0.2247 - val_accuracy: 0.9043
Epoch 919/1000
4/4 [==============================] - 2s 585ms/step - loss: 0.3135 - accuracy: 0.9345 - val_loss: 0.2205 - val_accuracy:

Epoch 970/1000
4/4 [==============================] - 2s 563ms/step - loss: 0.3262 - accuracy: 0.9339 - val_loss: 0.2887 - val_accuracy: 0.9026
Epoch 971/1000
4/4 [==============================] - 2s 596ms/step - loss: 0.2309 - accuracy: 0.9394 - val_loss: 0.2526 - val_accuracy: 0.9059
Epoch 972/1000
4/4 [==============================] - 2s 575ms/step - loss: 0.2081 - accuracy: 0.9372 - val_loss: 0.3141 - val_accuracy: 0.8944
Epoch 973/1000
4/4 [==============================] - 2s 600ms/step - loss: 0.2345 - accuracy: 0.9345 - val_loss: 0.2135 - val_accuracy: 0.9092
Epoch 974/1000
4/4 [==============================] - 2s 562ms/step - loss: 0.1681 - accuracy: 0.9438 - val_loss: 0.2260 - val_accuracy: 0.9043
Epoch 975/1000
4/4 [==============================] - 2s 570ms/step - loss: 0.2968 - accuracy: 0.9350 - val_loss: 0.2208 - val_accuracy: 0.9092
Epoch 976/1000
4/4 [==============================] - 2s 575ms/step - loss: 0.2344 - accuracy: 0.9356 - val_loss: 0.2200 - val_accuracy:

In [63]:
model.evaluate([testx[:, 0], testx[:, 1]],testy[:])

19/19 [==============================] - 1s 73ms/step - loss: 0.2055 - accuracy: 0.9142


[0.2054695039987564, 0.9141914248466492]

In [80]:
model.save("voice_siamese.h5")

In [64]:
def audio2vector(file_path, duration=3, sr=16000):
    input_length=sr*duration
    y, sr = librosa.load(file_path,sr=sr, duration=duration)
    dur = librosa.get_duration(y=y)
    if (round(dur) < duration):
        y = librosa.util.fix_length(y, round(input_length))      
    ps = librosa.feature.melspectrogram(y=y, sr=sr)
    return ps

In [95]:
voice1 = audio2vector("test/8000.wav")
voice1 = np.expand_dims(voice1,axis=0)

#voice2 = audio2vector("test/Benjamin_Netanyau/6.wav")
voice2 = audio2vector("test/8000.wav")

voice2 = np.expand_dims(voice2,axis=0)

pred = model.predict([voice1,voice2])[0]
print(pred)

[0.7493953]
